In [9]:
import json
import pickle
import os
import gc

import faiss
import numpy as np
from sentence_transformers import SentenceTransformer


In [10]:
with open("processed-data/judgment_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

texts = [c["text"] for c in chunks]
metadata = [c["metadata"] for c in chunks]

print(f"✅ Loaded {len(texts)} judgment chunks")


✅ Loaded 2018 judgment chunks


In [11]:
embedding_model = SentenceTransformer(
    "all-MiniLM-L6-v2",
    device="cpu"
)

print("✅ Embedding model loaded")


✅ Embedding model loaded


In [12]:
embeddings = embedding_model.encode(
    texts,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True
)

print("Embeddings shape:", embeddings.shape)


Batches: 100%|██████████| 127/127 [01:28<00:00,  1.43it/s]

Embeddings shape: (2018, 384)


In [13]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

index.add(embeddings)

print(f"✅ FAISS index created with {index.ntotal} vectors")


✅ FAISS index created with 2018 vectors


In [14]:
os.makedirs("vector-store", exist_ok=True)

faiss.write_index(index, "vector-store/faiss.index")

with open("vector-store/metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)

print("✅ FAISS index and metadata saved")


✅ FAISS index and metadata saved


In [15]:
del embeddings
gc.collect()

print("🧹 Memory cleaned")


🧹 Memory cleaned


In [16]:
index = faiss.read_index("vector-store/faiss.index")

with open("vector-store/metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

print("✅ FAISS index reloaded")


✅ FAISS index reloaded


In [17]:
def retrieve_precedents(query, top_k=5):
    query_embedding = embedding_model.encode(
        [query],
        convert_to_numpy=True
    )

    distances, indices = index.search(query_embedding, top_k)

    return [
        {
            "case_name": metadata[idx]["case_name"],
            "chunk_id": metadata[idx]["chunk_id"],
            "text": texts[idx][:400]
        }
        for idx in indices[0]
    ]


In [21]:
retrieve_precedents(
    "procedure established by law under Article 21",
    top_k=3
)


[{'case_name': 'Justice_K_S_Puttaswamy_Retd_And_Anr_vs_Union_Of_India_And_Ors_on_24_August_2017.PDF',
  'chunk_id': 550,
  'text': 'animate the content of Article 21. Second, the expression ‘procedure established by law’ in Article 21\ndoes not connote a formalistic requirement of a mere presence of procedure in enacted law. That\nexpression has been held to signify the content of the procedure and its quality which must be fair,\njust and reasonable. The mere fact that the law provides for the deprivation of life or personal lib'},
 {'case_name': 'Maneka_Gandhi_vs_Union_Of_India_on_25_January_1978.PDF',
  'chunk_id': 179,
  'text': 'endorsement, and a passport once granted may later be revoked or impounded. But the mere\nprescription of some kind of procedure cannot ever meet the mandate of article\n21. The procedure prescribed by law has to be fair, just and reasonable, not fanciful, oppressive or\narbitrary. The question whether the procedure prescribed by a law which curtails or ta

In [20]:
query = "procedure established by law and Article 21 personal liberty"

results = retrieve_precedents(query)

for r in results:
    print("\n📌 Case:", r["case_name"])
    print("🔹 Snippet:", r["text"])



📌 Case: Maneka_Gandhi_vs_Union_Of_India_on_25_January_1978.PDF
🔹 Snippet: procedure for depriving a person of 'personal liberty' and there is consequently no infringement of
the fundamental right conferred by Article 21, such law, in so far as it abridges or takes away any
fundamental right under Article 19 would have to meet the challenge of that article. This proposition
can no longer be disputed after the decisions in R. C. Cooper's case, Shambhu Nath Sarkar's case
a

📌 Case: Justice_K_S_Puttaswamy_Retd_And_Anr_vs_Union_Of_India_And_Ors_on_24_August_2017.PDF
🔹 Snippet: a right to life or personal liberty: Article 21 assumes or recognizes the fact that those
rights exist and affords protection against the deprivation of those rights to the extent
there provided. The expression “procedure established by law” does not mean merely
a procedural law but must also include substantive laws. The word “law” must mean a
valid law, that is, a law within the legislative comp

📌 Case: Maneka_Gan